# Colorspace - Colourflows - Volumetric Graphs

Create png files based on the screen dump from the atari800 emulation of colourspace.

Download our modified version of the atari800 emulator. This will allow us to create dump files of screen memory at specified trace points while colourspace is running.

In [62]:
import os
! rm -rf atari800
! git clone https://github.com/mwenge/atari800.git
os.chdir('atari800')
! git checkout presets
! ./autogen.sh
! ./configure --enable-monitormemorywatch
! make -j5
os.chdir('..')


Cloning into 'atari800'...
remote: Enumerating objects: 17545, done.
remote: Counting objects: 100% (1506/1506), done.
remote: Compressing objects: 100% (485/485), done.
remote: Total 17545 (delta 1067), reused 1419 (delta 1018), pack-reused 16039
Receiving objects: 100% (17545/17545), 5.75 MiB | 3.53 MiB/s, done.
Resolving deltas: 100% (13510/13510), done.
Branch 'presets' set up to track remote branch 'presets' from 'origin'.
Switched to a new branch 'presets'
configure.ac:204: installing './compile'
configure.ac:73: installing './install-sh'
configure.ac:73: installing './missing'
src/Makefile.am: installing './depcomp'

Now you need to run the configure script.  The configure script may take the
"--target=TARGET" option and various "--enable-FEATURE" and "--with-PACKAGE"
options.

Run "./configure --help" to see all available options.
Run "./configure --help=short" just to see the Atari800 FEATURE options.
Run "./configure --target=help" without a parameter to see the valid targets

-------------------------------------------------------
                 CONFIGURATION RESULTS:
-------------------------------------------------------
Host OS...............................: linux
Target ...............................: default

Interface for video...................: sdl
Using screenshots?....................: yes
    Supported screenshot formats......: pcx png
Using cycle exact?....................: yes
Using the very slow computer support?.: no
Using the crash menu?.................: yes
Using the paged attribute array?......: no
Using per opcode cycles update?.......: no
Using the buffered log?...............: no
Using Altirra BIOS ROM?...............: yes
Using the monitor assembler?..........: yes
Using code breakpoints and history?...: yes
Using user-defined breakpoints?.......: no
Using monitor hints?..................: yes
Using 6502 opcode profiling?..........: no
Using TRACE monitor command?..........: no
Using readline support in monitor?....: yes
Using UT

Run the atari800 emulator and write the screen memory dumps to a file.

In [30]:
import os
os.chdir('../psychedelia')
! make colourspace_nodemo.xex
os.chdir('../notebooks')
# 4327 is the point at which we've finished a round of painting
! ./atari800/src/atari800 -memwatchbpc 4327 -memwatchfile colorspace_colourflow_volumetric.txt ../psychedelia/bin/colourspace.xex

patch src/atari800/colourspace.asm -o src/atari800/colourspace_nodemo.asm < disable_demo.patch
patching file src/atari800/colourspace_nodemo.asm (read from src/atari800/colourspace.asm)
Hunk #1 succeeded at 4044 (offset 18 lines).
64tass -Wall -Wno-implied-reg --atari-xex -o bin/colourspace.xex -L bin/list-co1.txt -l bin/labels.txt src/atari800/colourspace_nodemo.asm
64tass Turbo Assembler Macro V1.56.2625?
64TASS comes with ABSOLUTELY NO WARRANTY; This is free software, and you
are welcome to redistribute it under certain conditions; See LICENSE!

Assembling file:   src/atari800/colourspace_nodemo.asm
Assembling file:   src/atari800/constants.asm
Assembling file:   src/atari800/patterns.asm
Assembling file:   src/atari800/presets.asm
Assembling file:   src/atari800/foreground.asm
Error messages:    None
Warning messages:  None
Passes:            3
Memory range:      $1f00-$7c66   $5d67
# the original xex file has an incorrect end-byte which we need to patch here.
dd if=bin/patch-atari

Read in the ram history from our dump file.

In [1]:
log_file = "colorspace_colourflow_volumetric.txt"
input_file = open(log_file,'r')

flatten = lambda l: [e for sublist in l for e in sublist]

ram_history = []
lines = input_file.readlines()
for i in range(1, len(lines), 132):
    # Screen RAM (7000-8000)
    raw_ram = [l[6:126].split() for l in lines[i:i+104]]
    pixel_ram = [int(v,16) for l in raw_ram for v in l][:0x1000]
    # Display list (8000-9000)
    raw_ram = [l[6:126].split() for l in lines[i+103:i+127]]
    display_list = [int(v,16) for l in raw_ram for v in l]
    # Color Registers
    color_registers = [int(i,16) for i in lines[i+129].split()[1:10]]
    # Color and Ooze data
    ooze_registers = [int(i,16) for i in lines[i+130].split()[1:30]]
    ram_history += [(pixel_ram, color_registers, display_list, ooze_registers)]

Helper functions for implementing the display list entries

In [2]:
def getByteForAddress(address,scr):
    if 0x7000 <= address < 0x8000:
        return scr[address - 0x7000]
    return 0

def drawBlankLines(pixels, byte, y, num, scr):
    #pixels[x, y] = ImageColor.getrgb('#000000')
    return num

def drawScanLine(pixels, byte, y, num, scr):
    for x in range(0,40):
        byte = getByteForAddress(address+x, scr)
        left_nibble = byte >> 4 & 0x0F 
        right_nibble = byte & 0x0F
        for n in range(0,4):
            pixels[y][(x*8)+n] = col_register[left_nibble]
        for n in range(0,4):
            pixels[y][(x*8)+n+4] = col_register[right_nibble]
    return 1

dlist_dict = {
    0x70: (drawBlankLines,1,1),
    0x90: (drawBlankLines,1,1),
    0x46: (drawBlankLines,3,1),
    0x4F: (drawScanLine,  3,1),
}
def display_list_entry(dl):
    # Consume entries in the display list one by one.
    i = 0
    while i < len(dl):
        b = dl[i]
        if b not in dlist_dict:
            yield (None,None,None,None,None)
        draw, advance, lines = dlist_dict[b]
        address = (dl[i+2]<<8 | dl[i+1]) if advance > 1 else 0
        i += advance
        yield (b, draw, address, lines,i)


Read in the ram  history and paint a png file using the display lists and the pixel data.

In [3]:
ROWS = 192
COLS = 320

patterns = []
pixel_history = []
pcol = []
#selected = [ram_history[x] for x in [188,236]]
for j, (scr, col_register, display_list, ooze) in enumerate(ram_history):
    col = col_register[:-1]
    pixels = [[0 for i in range(0,COLS)] for i in range(0,ROWS)]
    dlist = display_list_entry(display_list)
    x,y = 0,0
    while True:
        token, draw, address, num, i = next(dlist)
        if draw == None:
            break
        #print(hex(token),hex(address),num,i,display_list[i:i+2],len(display_list))
        y += draw(pixels, address, y, num, scr)
    if pcol and col != pcol:
        patterns += [(pcol,pixel_history)]
        pixel_history = []
    pcol = col
    pixel_history += [pixels]
patterns += [(pcol,pixel_history)]

len(patterns)

251

In [ ]:
l = [(i[1],sum([1 if r > 1 else 0 for s in p for l in s for r in l])) 
     for (i,p) in patterns]
l.sort(key=lambda x: x[1])
l

In [4]:
def firstNonZero(l):
    for i,e in enumerate(l):
        if e > 0:
            return i
    return i

def getLeftMostPixelForPattern(pattern):
    return min([firstNonZero(r) for scr in pattern for r in scr])

def getRightMostPixelForPattern(pattern):
    return len(pattern[0][0]) - min([firstNonZero(list(reversed(r)))
                     for scr in pattern for r in scr])

def rotatePatternToCols(pattern):
    pattern_cols = []
    for scr in pattern:
        col_arr = [[0 for i in range(len(scr))] for j in range(len(scr[0]))]
        for i,r in enumerate(scr):
            for j,v in enumerate(r):
                col_arr[j][i] = v
        pattern_cols += [col_arr]
    return pattern_cols

def cullPattern(pattern):
    first_left = getLeftMostPixelForPattern(pattern)
    last_right = getRightMostPixelForPattern(pattern)
    width = last_right - first_left

    pattern_as_cols = rotatePatternToCols(pattern)
    first_top = getLeftMostPixelForPattern(pattern_as_cols)
    last_bottom = getRightMostPixelForPattern(pattern_as_cols)
    height = last_bottom - first_top
    
    culled = [[r[first_left:last_right] 
              for r in scr[first_top:last_bottom]]
              for scr in pattern]
    return culled

#cullPattern(patterns[0])
#patterns[0]

In [5]:
from PIL import Image, ImageColor, ImageChops
from matplotlib.colors import LightSource
import colorspace_colors as cc
import numpy as np

def explode(data):
    size = np.array(data.shape)*2
    data_e = np.zeros(size - 1, dtype=data.dtype)
    data_e[::2, ::2, ::2] = data
    return data_e

def createVoxelData(screens):
    # prepare some coordinates
    width = len(screens[0][0])
    height = len (screens[0])
    x, y, z = np.indices((width,height,len(screens)))
    voxelarray = np.zeros((width,height,len(screens)), dtype=bool)
    colorarray = np.empty(voxelarray.shape, dtype=object)
    for zp,screen in enumerate(screens):
        for yp, row in enumerate(list(reversed(screen))):
            for xp, col in enumerate(row):
                # Skip black, white and gray cells
                if col in [0,1,0x0c]:
                    continue
                cube = (x == xp) & (y == yp) & (z == zp)
                voxelarray |= cube
                color =  cc.color_value_to_html[col]
                colorarray[cube] = color+"c0"

    # upscale the above voxel image, leaving gaps
    filled_2 = explode(voxelarray)
    ecolors_2 = explode(colorarray)

    # Shrink the gaps
    x, y, z = np.indices(np.array(filled_2.shape) + 1).astype(float) // 2
    x[0::2, :, :] += 0.10
    y[:, 0::2, :] += 0.10
    z[:, :, 0::2] += 0.10
    x[1::2, :, :] += 0.90
    y[:, 1::2, :] += 0.90
    z[:, :, 1::2] += 0.90

    return ((x,y,z), filled_2, ecolors_2)

def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img

In [7]:
# Select our pattern
import os
import gc
import datetime
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,40)
plt.rcParams['figure.dpi'] = 40


def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    bbox = diff.getbbox()
    print(bbox)
    return im.crop(bbox)

for i, (pid_array,screens) in enumerate(patterns):
    for azim in [-45]:
        pid = int(pid_array[1])
        print(pid, datetime.datetime.now(), "checking")
        if os.path.isfile(f'colorspace_colourflow/colourflow_{pid}{azim}.png'):
            continue
        screens = cullPattern(screens)
        print(pid, datetime.datetime.now(), "createVoxelData")
        ((x,y,z), voxels, vcolors) = createVoxelData(screens)
        # and plot everything
        print(pid, datetime.datetime.now(), "plot")
        ax = plt.figure().add_subplot(projection='3d')
        ls = LightSource(45)
        ax.voxels(x,y,z,voxels, facecolors=vcolors, edgecolors=vcolors,lightsource=ls)
        ax.axis('off')
        ax.view_init(azim=azim, vertical_axis='y')
        print(pid, datetime.datetime.now(), "fig2img")
        img = fig2img(plt.gcf())
        img = img.resize((int(img.width / 2), int(img.height / 2)), Image.NEAREST)
        
        print(pid, datetime.datetime.now(), "remove alpha")
        newImage = []
        for item in img.getdata():
            if item[:3] == (255, 255, 255):
                newImage.append((255, 255, 255, 0))
            else:
                newImage.append(item)

        print(pid, datetime.datetime.now(), "write image")
        img.putdata(newImage)
        img = trim(img)
        img.save(f'colorspace_colourflow/colourflow_{pid}{azim}.png')
        plt.clf()
        del voxels, vcolors
        plt.close()
        gc.collect()


49 2024-02-10 13:18:08.582633 checking
50 2024-02-10 13:18:08.582900 checking
51 2024-02-10 13:18:08.582989 checking
52 2024-02-10 13:18:08.583087 checking
53 2024-02-10 13:18:08.583161 checking
54 2024-02-10 13:18:08.583237 checking
55 2024-02-10 13:18:08.583367 checking
56 2024-02-10 13:18:08.583444 checking
57 2024-02-10 13:18:08.583531 checking
58 2024-02-10 13:18:08.583607 checking
59 2024-02-10 13:18:08.583695 checking
60 2024-02-10 13:18:08.583775 checking
61 2024-02-10 13:18:08.583836 checking
62 2024-02-10 13:18:08.583897 checking
63 2024-02-10 13:18:08.583955 checking
64 2024-02-10 13:18:08.584008 checking
65 2024-02-10 13:18:08.584063 checking
65 2024-02-10 13:18:09.530241 createVoxelData
65 2024-02-10 13:18:12.885183 plot
65 2024-02-10 13:19:09.528454 fig2img
65 2024-02-10 13:19:12.591431 remove alpha
65 2024-02-10 13:19:12.814425 write image
(223, 189, 659, 603)
66 2024-02-10 13:19:20.186452 checking
66 2024-02-10 13:19:20.926081 createVoxelData
66 2024-02-10 13:19:22.8870

93 2024-02-10 13:40:15.676460 write image
(223, 189, 659, 603)
94 2024-02-10 13:40:23.055480 checking
94 2024-02-10 13:40:23.704719 createVoxelData
94 2024-02-10 13:40:25.462357 plot
94 2024-02-10 13:41:01.816456 fig2img
94 2024-02-10 13:41:03.909018 remove alpha
94 2024-02-10 13:41:04.155633 write image
(227, 190, 655, 601)
95 2024-02-10 13:41:10.415023 checking
95 2024-02-10 13:41:11.149896 createVoxelData
95 2024-02-10 13:41:12.817703 plot
95 2024-02-10 13:41:49.576115 fig2img
95 2024-02-10 13:41:51.626420 remove alpha
95 2024-02-10 13:41:51.899048 write image
(227, 190, 655, 601)
96 2024-02-10 13:41:58.410013 checking
96 2024-02-10 13:41:59.243923 createVoxelData
96 2024-02-10 13:42:01.145130 plot
96 2024-02-10 13:42:36.702315 fig2img
96 2024-02-10 13:42:38.787031 remove alpha
96 2024-02-10 13:42:39.034952 write image
(227, 190, 655, 601)
97 2024-02-10 13:42:45.397067 checking
97 2024-02-10 13:42:46.005680 createVoxelData
97 2024-02-10 13:42:46.928505 plot
97 2024-02-10 13:43:13.51

124 2024-02-10 13:59:43.833623 plot
124 2024-02-10 14:00:02.021824 fig2img
124 2024-02-10 14:00:03.168651 remove alpha
124 2024-02-10 14:00:03.453634 write image
(245, 199, 634, 590)
125 2024-02-10 14:00:09.689770 checking
125 2024-02-10 14:00:10.052517 createVoxelData
125 2024-02-10 14:00:10.338951 plot
125 2024-02-10 14:00:28.721308 fig2img
125 2024-02-10 14:00:29.837353 remove alpha
125 2024-02-10 14:00:30.057905 write image
(245, 199, 634, 590)
126 2024-02-10 14:00:36.047325 checking
126 2024-02-10 14:00:36.389262 createVoxelData
126 2024-02-10 14:00:36.689571 plot
126 2024-02-10 14:00:54.744340 fig2img
126 2024-02-10 14:00:55.893227 remove alpha
126 2024-02-10 14:00:56.118635 write image
(245, 199, 634, 590)
127 2024-02-10 14:01:02.742896 checking
127 2024-02-10 14:01:03.091549 createVoxelData
127 2024-02-10 14:01:03.325323 plot
127 2024-02-10 14:01:21.648976 fig2img
127 2024-02-10 14:01:22.825019 remove alpha
127 2024-02-10 14:01:23.101315 write image
(245, 199, 634, 590)
128 202

154 2024-02-10 14:19:36.678949 write image
(233, 193, 648, 597)
155 2024-02-10 14:19:44.205644 checking
155 2024-02-10 14:19:44.818574 createVoxelData
155 2024-02-10 14:19:45.820320 plot
155 2024-02-10 14:20:14.352122 fig2img
155 2024-02-10 14:20:16.528670 remove alpha
155 2024-02-10 14:20:16.897524 write image
(233, 193, 648, 597)
156 2024-02-10 14:20:24.764424 checking
156 2024-02-10 14:20:25.508243 createVoxelData
156 2024-02-10 14:20:26.922590 plot
156 2024-02-10 14:20:53.031836 fig2img
156 2024-02-10 14:20:54.591139 remove alpha
156 2024-02-10 14:20:54.805360 write image
(233, 193, 648, 597)
157 2024-02-10 14:21:00.670785 checking
157 2024-02-10 14:21:01.204618 createVoxelData
157 2024-02-10 14:21:02.142614 plot
157 2024-02-10 14:21:26.716671 fig2img
157 2024-02-10 14:21:28.345373 remove alpha
157 2024-02-10 14:21:28.567895 write image
(233, 193, 648, 597)
158 2024-02-10 14:21:34.338941 checking
158 2024-02-10 14:21:34.857002 createVoxelData
158 2024-02-10 14:21:35.803433 plot
158

185 2024-02-10 14:42:05.642476 createVoxelData
185 2024-02-10 14:42:07.516794 plot
185 2024-02-10 14:42:43.576667 fig2img
185 2024-02-10 14:42:45.704028 remove alpha
185 2024-02-10 14:42:45.976323 write image
(227, 190, 655, 601)
186 2024-02-10 14:42:53.125898 checking
186 2024-02-10 14:42:53.677470 createVoxelData
186 2024-02-10 14:42:54.650145 plot
186 2024-02-10 14:43:22.347828 fig2img
186 2024-02-10 14:43:24.054294 remove alpha
186 2024-02-10 14:43:24.335750 write image
(233, 193, 648, 597)
187 2024-02-10 14:43:31.698448 checking
187 2024-02-10 14:43:32.872736 createVoxelData
187 2024-02-10 14:43:37.679969 plot
187 2024-02-10 14:44:34.070257 fig2img
187 2024-02-10 14:44:37.593694 remove alpha
187 2024-02-10 14:44:37.845215 write image
(221, 188, 662, 604)
188 2024-02-10 14:44:45.166754 checking
188 2024-02-10 14:44:45.792169 createVoxelData
188 2024-02-10 14:44:46.749931 plot
188 2024-02-10 14:45:13.705389 fig2img
188 2024-02-10 14:45:15.371450 remove alpha
188 2024-02-10 14:45:15.

216 2024-02-10 15:15:36.884452 remove alpha
216 2024-02-10 15:15:37.166685 write image
(233, 193, 648, 597)
217 2024-02-10 15:15:44.303527 checking
217 2024-02-10 15:15:45.172876 createVoxelData
217 2024-02-10 15:15:48.238701 plot
217 2024-02-10 15:16:32.551540 fig2img
217 2024-02-10 15:16:35.222093 remove alpha
217 2024-02-10 15:16:35.484333 write image
(223, 189, 659, 603)
218 2024-02-10 15:16:42.673855 checking
218 2024-02-10 15:16:43.429327 createVoxelData
218 2024-02-10 15:16:45.210506 plot
218 2024-02-10 15:17:21.879158 fig2img
218 2024-02-10 15:17:23.963022 remove alpha
218 2024-02-10 15:17:24.232747 write image
(227, 190, 655, 601)
219 2024-02-10 15:17:31.356441 checking
219 2024-02-10 15:17:32.488900 createVoxelData
219 2024-02-10 15:17:37.081204 plot
219 2024-02-10 15:18:32.000634 fig2img
219 2024-02-10 15:18:35.434604 remove alpha
219 2024-02-10 15:18:35.703064 write image
(221, 188, 662, 604)
220 2024-02-10 15:18:42.980814 checking
220 2024-02-10 15:18:43.905256 createVoxel

247 2024-02-10 15:50:23.806377 checking
247 2024-02-10 15:50:25.189782 createVoxelData
247 2024-02-10 15:50:33.496413 plot
247 2024-02-10 15:51:46.694500 fig2img
247 2024-02-10 15:51:50.732791 remove alpha
247 2024-02-10 15:51:50.964163 write image
(215, 186, 666, 606)
248 2024-02-10 15:51:58.132036 checking
248 2024-02-10 15:51:59.181140 createVoxelData
248 2024-02-10 15:52:03.524017 plot
248 2024-02-10 15:52:56.989587 fig2img
248 2024-02-10 15:53:00.184651 remove alpha
248 2024-02-10 15:53:00.443506 write image
(221, 188, 662, 604)
249 2024-02-10 15:53:07.717225 checking
249 2024-02-10 15:53:09.028264 createVoxelData
249 2024-02-10 15:53:14.949137 plot
249 2024-02-10 15:54:17.861121 fig2img
249 2024-02-10 15:54:21.335665 remove alpha
249 2024-02-10 15:54:21.594398 write image
(223, 193, 664, 605)
250 2024-02-10 15:54:28.578635 checking
250 2024-02-10 15:54:30.052595 createVoxelData
250 2024-02-10 15:54:38.016368 plot
250 2024-02-10 15:55:51.011027 fig2img
250 2024-02-10 15:55:54.8898

22 2024-02-10 16:39:19.253011 createVoxelData
22 2024-02-10 16:39:34.527298 plot
22 2024-02-10 16:41:14.857988 fig2img
22 2024-02-10 16:41:20.741764 remove alpha
22 2024-02-10 16:41:20.991018 write image
(205, 185, 668, 607)
23 2024-02-10 16:41:29.166857 checking
23 2024-02-10 16:41:30.639598 createVoxelData
23 2024-02-10 16:41:40.352900 plot
23 2024-02-10 16:42:53.945840 fig2img
23 2024-02-10 16:43:04.627450 remove alpha
23 2024-02-10 16:43:04.867839 write image
(215, 186, 666, 606)
24 2024-02-10 16:43:11.794771 checking
24 2024-02-10 16:43:12.968440 createVoxelData
24 2024-02-10 16:43:20.533487 plot
24 2024-02-10 16:44:27.504131 fig2img
24 2024-02-10 16:44:31.824789 remove alpha
24 2024-02-10 16:44:32.114730 write image
(219, 187, 664, 605)
25 2024-02-10 16:44:39.375118 checking
25 2024-02-10 16:44:40.969486 createVoxelData
25 2024-02-10 16:44:49.965318 plot
25 2024-02-10 16:46:06.567381 fig2img
25 2024-02-10 16:46:17.468153 remove alpha
25 2024-02-10 16:46:17.708597 write image
(215